In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F


device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [2]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data',
    train = True,
    download = True,
    transform = transforms.Compose([
      transforms.ToTensor()
    ]
    )
)



  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
ZeroToNine = [[0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,1], [0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,1,0,0], [0,0,0,0,0,1,0,0,0], [0,0,0,0,1,0,0,0,0], [0,0,0,1,0,0,0,0,0], [0,0,1,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,0,0]]
ZeroToEighteen = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]                  
]

#print(ZeroToNine[0])
#print(ZeroToNine[9])
#print(ZeroToEighteen[0])
#print(ZeroToEighteen[18])

In [ ]:
from torch.utils.data import Dataset
import random


class MyDataset(Dataset):
  def __init__(self, train_set):
    self.data=[]
    for tr in train_set:
      image, label = tr
      rand = random.randint(0, 9)
      a=ZeroToNine[rand]
      
      b = ZeroToEighteen[label + rand]
      final_value = a + b
      
      self.data.append((image, rand, final_value))
      

  def __getitem__(self, index):
    return self.data[index]

  def __len__(self):
    return len(self.data)


myDataset = MyDataset(train_set)

for m in myDataset:
  print(m)
  break

In [5]:
import torch.nn as nn

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels =1, out_channels=6, kernel_size = 5)
    self.conv2 = nn.Conv2d(in_channels =6, out_channels=12, kernel_size = 5)
    self.fc1 = nn.Linear(in_features= 192, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features = 60)
    self.out = nn.Linear(in_features=60, out_features=27 )

  def forward(self, t, u):
    print(t)
    print(u)
    x = t
    y = ZeroToNine[u]
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size = 2, stride = 2)
    
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size = 2, stride = 2)

    #print(x.shape)
    x= x.reshape(-1, 12 * 4 * 4)
    #print(x.shape)
    x = x + u
    #print(x.shape)
    #print(x)
    x = self.fc1(x)
    x = F.relu(x)

    x = self.fc2(x)
    x = F.relu(x)

    x = self.out(x)

    ## x = F.softmax(x, dim =1)


    return x

In [6]:
def get_num_correct(preds, sums):
  return preds.argmax(dim=1).eq(sums).sum().item()

In [8]:
network = Network()
print(network)
print(network.parameters())

train_loader = torch.utils.data.DataLoader(
    myDataset,
    batch_size = 32,
    shuffle = True
)

optimizer = optim.Adam(network.parameters(), lr = 0.01)
total_loss = 0
total_correct = 0

for epoch in range(10):
  for batch in train_loader:
    images, rands, sums = batch

  
    #print(images)
    #print(rands)
    #print(sums)
    preds = network(images, rands)
    #print(preds)
    #print(preds.shape)
    sum = torch.tensor(sums)

    #sum = sum.reshape(1,27)
    #print(sum)
    #print(sum.shape)
    loss = F.cross_entropy(pred, sum)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds, sums)

  print(
      "Epoch :", epoch, 
      "total_correct : ", total_correct,
      "total_loss : ", total_loss
  )
  

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=27, bias=True)
)
<generator object Module.parameters at 0x7fb1a7a61bd0>
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
      

TypeError: ignored

In [ ]:
for epoch in range(10):
  for batch in train_loader:
    images, labels = batch

    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimzer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

    print(
        "Epoch :", epoch, 
        "total_correct : ", total_correct,
        "total_loss : ", total_loss
    )
  

NameError: ignored